In [1]:
import os
import sys
#sys.path.insert(0, os.path.dirname(os.getcwd()))
import pickle

import numpy as np
import torch
#import torch.cuda
import timeit
import pandas as pd
#import argparse
import itertools
from load_data import load_data # segment_SHAP
from evaluation.metrics.AUC_difference import AUIDC_metric
from models.predictor_utils import load_predictor
from pickle import dump



In [2]:
# PATHS
datasets_folder_path = None
attributions_folder_path = None
trained_models_folder_path = None
save_results_folder_path = None

In [3]:
# settings
dataset_names = ['UWAVE']    #[sys.argv[1]] # ['UWAVE', "KeplerLightCurves", "MP8", "gunpoint", "EOG"] 
predictor_names = ['resNet']    #[sys.argv[2]] ["randomForest", 'miniRocket', 'resNet', "QUANT"]
segmentation_names = ["clasp","greedygaussian", "equal", "infogain","nnsegment"] # ["clasp","greedygaussian", "equal", "infogain","nnsegment"] 
background_names =  ["average", "zero","sampling"] #["average", "zero", "sampling"]
normalization_names = ["default", "normalized"]

metric_names = ["AUC_difference"]

demo_mode = True
demo_mode_samples = 10
# demo
if demo_mode:
    dataset_names = ["KeplerLightCurves"]
    predictor_names = ["randomForest", 'miniRocket', 'resNet', "QUANT"]
    segmentation_names = ["clasp","greedygaussian", "equal", "infogain","nnsegment"]
    background_names = ["average", "zero","sampling"] #, 'zero','sampling']
    normalization_names = ["default", "normalized"]

# optional
# # get infos about which explanations are evaluated
# datasets = list( explanations['attributions'].keys() )
# segmentations = list( explanations['attributions'][datasets[0]].keys() )
# predictors = list( explanations['attributions'][datasets[0]][segmentations[0]].keys() )
# backgrounds = list( explanations['attributions'][datasets[0]][segmentations[0]][predictors[0]].keys() )
# result_types = ['default','normalized']

In [4]:
# column_names = ['Dataset', 'Segmentation', 'ML model', 'Background', 'Normalization', 'Metric', 'Perturb', "Result"]
# df = pd.DataFrame(data=data_list, columns = column_names)

In [5]:
# df

In [6]:
cwd = os.getcwd()
if datasets_folder_path is None:
    datasets_folder_path = "datasets" #os.path.join(cwd, "datasets")
if attributions_folder_path is None:
    attributions_folder_path = "attributions"
if trained_models_folder_path is None:
    trained_models_folder_path = "trained_models"
if save_results_folder_path is None:
    save_results_folder_path = os.path.join("evaluation", "results")

# device
device = "cuda" if torch.cuda.is_available() else "cpu"

starttime = timeit.default_timer()

eval_metrics = dict.fromkeys(metric_names)
for key in eval_metrics:
    if key=="AUC_difference":
        eval_metrics[key] = AUIDC_metric()
    else:
        raise KeyError(f"key {key} has no corresponding eval metric defined")

for dataset_name in dataset_names:
    print("Dataset: ", dataset_name)
    # loading dataset
    X_train, X_test, y_train, y_test, enc = load_data(subset='all', dataset_name=dataset_name, path=datasets_folder_path)
    if demo_mode:
        n_test_samples = X_test.shape[0]
        n_samples_to_choose = np.min([n_test_samples, demo_mode_samples])
        demo_mode_idxs = np.random.choice(n_test_samples, demo_mode_samples, replace=False)

        X_test, y_test = X_test[demo_mode_idxs], y_test[demo_mode_idxs]

    for eval_metric in eval_metrics.values():
        eval_metric.fit_data(X_train, X_test, y_train, y_test)

    for predictor_name in predictor_names:
        print("Predictor: ", predictor_name)

        # load classifier
        predictor = load_predictor(path=trained_models_folder_path, predictor_name=predictor_name, dataset_name=dataset_name, device=device) # torch.device(device)

        # load explanations
        attribution_filename = "_".join(("all_results", dataset_name, predictor_name)) + ".npy"
        explanations = np.load(os.path.join(attributions_folder_path, attribution_filename), allow_pickle=True).item() # FileNotFoundError
        label_mapping = explanations['label_mapping'][dataset_name]

        for eval_metric in eval_metrics.values():
            eval_metric.fit_ml_model(predictor)

        data_list = []

        for key in itertools.product(segmentation_names, background_names, normalization_names, metric_names):
            segmentation_name, background_name, normalization_name, metric_name = key
            print("assessing", key)

            # load model and explanations to access
            try:
                attributions = explanations['attributions'][dataset_name][segmentation_name][predictor_name][background_name][normalization_name]
            except KeyError as error:
                print('Warning: attributions is missing keys, skipping to next ' + repr(error))
                continue
            try:
                y_test_pred = explanations['y_test_pred'][dataset_name][predictor_name]
            except KeyError:
                y_test_pred = explanations['y_test_pred'][dataset_name][segmentation_name][predictor_name]

            eval_metric = eval_metrics[metric_name]
            results = eval_metric.evaluate(attributions, y_test_pred, predictor)

            for result_tuple in results:
                data_list.append((dataset_name, segmentation_name, predictor_name, background_name, normalization_name, metric_name) + result_tuple)

            print("elapsed time", (timeit.default_timer() - starttime))

        # save
        column_names = ['Dataset', 'Segmentation', 'ML model', 'Background', 'Normalization', 'Metric', 'Perturb', "Result"]
        df = pd.DataFrame(data=data_list, columns = column_names)
        file_name = "_".join(("evaluation", predictor_name, dataset_name))
        result_path = os.path.join(save_results_folder_path, file_name)
        if not demo_mode:
            df.to_csv(result_path)
            # with open( "_".join( (dataset_name,classifier_name)) ,"wb") as f:
            #     pickle.dump(results_dict, f)

Dataset:  UWAVE
Predictor:  resNet
assessing ('nnsegment', 'average', 'default', 'AUC_difference')
elapsed time 9.232853599998634
assessing ('nnsegment', 'average', 'normalized', 'AUC_difference')
elapsed time 17.830368999988423


In [7]:
df

,Dataset,Segmentation,ML model,Background,Normalization,Metric,Perturb,Result
0,UWAVE,nnsegment,resNet,average,default,AUC_difference,default,0.342571
1,UWAVE,nnsegment,resNet,average,normalized,AUC_difference,default,0.347857
